In [1]:
import flywheel
import flywheel_gear_toolkit as gt
from pprint import pprint

In [35]:
fw = flywheel.Client()

In [3]:
user_info = fw.get_current_user()
print(f"Firstname: {user_info.firstname} \n"
      f"Lastname: {user_info.lastname} \n"
      f"Email: {user_info.email}")

Firstname: Xuchu 
Lastname: Liu 
Email: Xuchu_Liu@rush.edu


In [11]:
project_sessions = fw.sessions()
for session in project_sessions:
    print(session.label)

2004-08-17 10_58_48
07-03-08 11_43 AM
07-12-04 12_34 PM
09-18-07 9_27 AM
09-01-05 9_18 AM
OHIF_6
OHIF_11
exam_000007


In [24]:
my_session_label = 'OHIF_6'
my_session_id = fw.sessions.find(f'label={my_session_label}')[0].id
print(my_session_id)

61f008e591188d92013e6c34


In [20]:
my_session = fw.get_session(my_session_id)

In [43]:
my_group = my_session.parents.group

In [47]:
my_metadata = my_session.info
ohif_metadata = my_metadata['ohifViewer']

In [48]:
pprint(ohif_metadata['measurements']['Length'])

[]


In [54]:
Pons_Length = {}
Pons_Length['description'] = 'A-P Diameter of Pons'

In [55]:
Vermis_Length = {}
Vermis_Length['description'] = 'A-P Diameter of Vermis'

In [56]:
HVermis_Length = {}
HVermis_Length['description'] = 'Height of Vermis'

In [57]:
Fronto_Length = {}
Fronto_Length['description'] = 'Fronto-Occpitial'

In [58]:
ohif_metadata['measurements']['Length'].append(Pons_Length)
ohif_metadata['measurements']['Length'].append(Vermis_Length)
ohif_metadata['measurements']['Length'].append(HVermis_Length)
ohif_metadata['measurements']['Length'].append(Fronto_Length)
# ohif_metadata['measurements']['Length'] = [] # delete

In [59]:
my_session.update_info(my_metadata)

In [25]:
my_acquisition = my_session.acquisitions()
for acquisition in my_acquisition:
    for file in acquisition.files[:1]:
        if file.type == 'dicom':
            print(file.name)

1 - Scout Expiration BH.dicom.zip
10 - COR Body TRUEFIS BH.dicom.zip
11 - AXIAL BRAIN TRUEFIS BH4MM.dicom.zip
12 - AXIAL BRAIN TRUEFIS BH4MM.dicom
13 - AXIAL BRAIN TRUEFIS BH4MM.dicom.zip
14 - Scout Expiration BH.dicom.zip
15 - AX BodyTRUEFIS  BH.dicom.zip
16 - COR BodyTRUEFIS BH.dicom.zip
17 - COR Body TRUEFIS BH.dicom.zip
18 - COR BodyTRUEFIS BH.dicom.zip
2 - SAG BrainT2 HASTE BH.dicom.zip
3 - AX BrainT2 HASTE BH.dicom.zip
4 - COR BrainT2 HASTE BH.dicom.zip
5 - Scout Expiration BH.dicom.zip
6 - Sag Spine T2 HASTE BH.dicom.zip
7 - SAG BrainT2 HASTE BH.dicom.zip
8 - Sag Spine T2 HASTE BH.dicom.zip
9 - COR Spine T2 HASTE BH.dicom.zip


In [42]:
context = gt.GearToolkitContext()
context.config_from_args([])

{'config': {'ip': '127.0.0.1',
  'stdin': '9003',
  'control': '9001',
  'hb': '9000',
  'Session.signature_scheme': '"hmac-sha256"',
  'Session.key': 'b"10c03b03-bae0-4dfe-b1d1-9f04004cb388"',
  'shell': '9002',
  'transport': '"tcp"',
  'iopub': '9004'},
 'inputs': {'f': {'base': 'file',
   'hierarchy': {'id': 'aex', 'type': 'acquisition'},
   'location': {'name': 'tmp-25430780eeTJ4mFqikO.json',
    'path': '/tmp/tmp-25430780eeTJ4mFqikO.json'},
   'object': {'classification': {'Intent': [], 'Measurement': []},
    'info': {},
    'measurements': [],
    'mimetype': '',
    'modality': '',
    'size': 261,
    'tags': [],
    'type': ''}}},
 'destination': {'id': 'aex', 'type': 'acquisition'}}